In [ ]:
import os
from typing import TypedDict, List, Dict, Annotated
import operator

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

from langgraph.graph import StateGraph, END



The goal of  our project was if we were able to build an agentic system that could produce books, upload them some marketplace ( for us this was Amazon) and if we could somewhat automate the process of writing book. I personally have private interest in literature so I really wonder if this is sensible and if it truly makes sense to automate this process. 

The idea of the workflow is simple.  
The user query is taken. 
Then an outline is generated. 
This outline is then used in a loop to generate the individual chapters.
Then the whole book is concatenated and the book automatically uploaded

In [ ]:
# --- 1. Define the State ---
# This state is simplified to focus on the narrative structure: premise -> outline -> chapters.

class BookWriterState(TypedDict):
    """
    Represents the state of our book writing graph.

    Attributes:
        book_topic: The initial high-level topic for the book.
        book_premise: A more detailed premise or summary generated by the LLM.
        book_outline: A dictionary containing the title and a list of chapter titles.
        chapter_index: An index to keep track of the current chapter being written.
        draft_chapters: A list to store the written content of each chapter.
        error: A field to store any errors that occur.
    """
    book_topic: str
    book_premise: str
    book_outline: Dict
    chapter_index: int
    # We use Annotated and operator.add to accumulate chapters in the list.
    draft_chapters: Annotated[List[str], operator.add]
    error: str




llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0.2, max_output_tokens=1024)
json_parser = JsonOutputParser()
str_parser = StrOutputParser()

# --- 3. Define the Nodes (The "Workers") ---

def generate_premise_node(state: BookWriterState) -> Dict:
    """Generates a compelling book premise from a high-level topic."""
    print("--- Generating Book Premise ---")
    try:
        prompt = PromptTemplate(
            template="Generate a compelling one-paragraph book premise based on this topic: {topic}",
            input_variables=["topic"],
        )
        chain = prompt | llm | str_parser
        premise = chain.invoke({"topic": state["book_topic"]})
        print(f"Generated Premise: {premise}")
        return {"book_premise": premise}
    except Exception as e:
        return {"error": f"Failed to generate premise: {e}"}

def generate_outline_node(state: BookWriterState) -> Dict:
    """Generates a book outline based on the premise."""
    print("\n--- Generating Book Outline ---")
    try:
        prompt = PromptTemplate(
            template="Based on this premise, create a book outline with a title and a list of chapter titles (JSON format). Premise: {premise}",
            input_variables=["premise"],
        )
        chain = prompt | llm | json_parser
        outline = chain.invoke({"premise": state["book_premise"]})
        print(f"Generated Outline: {outline}")
        return {"book_outline": outline}
    except Exception as e:
        return {"error": f"Failed to generate outline: {e}"}

def write_chapter_node(state: BookWriterState) -> Dict:
    """Writes the content for a single chapter based on the outline."""
    print("\n--- Writing Chapter ---")
    try:
        outline = state["book_outline"]
        chapter_index = state["chapter_index"]
        chapter_title = outline["chapters"][chapter_index]
        
        prompt = PromptTemplate(
            template="Write the content for Chapter {chapter_num}: '{chapter_title}'. The overall book premise is: {premise}",
            input_variables=["chapter_num", "chapter_title", "premise"],
        )
        chain = prompt | llm | str_parser
        chapter_content = chain.invoke({
            "chapter_num": chapter_index + 1,
            "chapter_title": chapter_title,
            "premise": state["book_premise"]
        })
        print(f"Wrote Content for: {chapter_title}")
        return {"draft_chapters": [chapter_content]}
    except Exception as e:
        return {"error": f"Failed to write chapter: {e}"}

# --- 4. Define the Conditional Edge ---

def should_continue_writing(state: BookWriterState) -> str:
    """Checks if there are more chapters to write."""
    print("--- Checking for More Chapters ---")
    next_index = state["chapter_index"] + 1
    
    if next_index < len(state["book_outline"]["chapters"]):
        print("Decision: Continue with next chapter.")
        # Update the index in the state to process the next chapter
        return "continue_writing"
    else:
        print("Decision: All chapters written. End process.")
        return "end_writing"

# --- 5. Assemble the Graph ---

workflow = StateGraph(BookWriterState)

# Add nodes
workflow.add_node("generate_premise", generate_premise_node)
workflow.add_node("generate_outline", generate_outline_node)
workflow.add_node("write_chapter", write_chapter_node)

# Define the flow
workflow.set_entry_point("generate_premise")
workflow.add_edge("generate_premise", "generate_outline")
workflow.add_edge("generate_outline", "write_chapter")

# Add the conditional edge for the chapter writing loop
workflow.add_conditional_edges(
    "write_chapter",
    should_continue_writing,
    {
        "continue_writing": "write_chapter", # If 'continue', loop back
        "end_writing": END                   # If 'done', finish
    }
)

# Compile the graph
app = workflow.compile(checkpointer=None) # A checkpointer is needed for state updates in conditional edges, but we can manage manually for this simple case.

# --- 6. Run the Graph ---

# We need a function to manually update the index before the next loop
def run_graph(initial_input):
    state = initial_input
    while True:
        output = app.invoke(state)
        # Check if the process has ended
        if "__end__" in output:
            return output
        
        # Manually update the chapter index for the next iteration
        state = output.copy()
        state["chapter_index"] = state.get("chapter_index", -1) + 1


initial_input = {
    "book_topic": "A space opera about a lost human colony that has evolved.",
    "chapter_index": 0,
    "draft_chapters": [],
}

print("--- Starting Book Writer Graph ---")
final_state = run_graph(initial_input)

print("\n--- Graph Finished ---")
print(f"\nTitle: {final_state.get('book_outline', {}).get('title')}")
print("\nPremise:")
print(final_state.get('book_premise'))
print("\nDraft Chapters:")
for i, chapter in enumerate(final_state.get('draft_chapters', [])):
    print(f"--- Chapter {i+1} ---\n{chapter}\n")